In [13]:
import nibabel
import nibabel.affines
from PIL import Image
import os
import fnmatch
import numpy as np
import ants
import scipy.io as sio
import pickle
import concurrent.futures

# Custom modules
from preprocessing.post_fastsurfer.subjects import *

ModuleNotFoundError: No module named 'preprocessing.post_fastsurfer.subjects'

In [6]:
data_path = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch disk/full-datasets/adni1-complete-1yr-3t"

In [8]:
find_subjects(data_path)

NameError: name 'find_subjects' is not defined